In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import numpy as np
import shutil

In [22]:
# Parameters
IMAGE_SIZE = (128, 128)  # Must match dataset image size
BATCH_SIZE = 32
EPOCHS = 50
DATASET_DIR = "dataset"   # Path to dataset folder (contains subfolders for each class)
SPLIT_DIR = "dataset_split"  # Temporary folder for train-validation-test split
NUM_CLASSES = len(os.listdir(DATASET_DIR))  # Number of gesture classes

# 1️⃣ Ensure the dataset_split directory is empty
if os.path.exists(SPLIT_DIR):
    shutil.rmtree(SPLIT_DIR)
os.makedirs(SPLIT_DIR, exist_ok=True)

# 2️⃣ Load Images & Split Dataset
train_dir = os.path.join(SPLIT_DIR, "train")
val_dir = os.path.join(SPLIT_DIR, "validation")
test_dir = os.path.join(SPLIT_DIR, "test")

for folder in os.listdir(DATASET_DIR):  # Loop through each class
    class_path = os.path.join(DATASET_DIR, folder)
    images = os.listdir(class_path)

    # Split into train (80%), validation (10%), test (10%)
    train_imgs, test_imgs = train_test_split(images, test_size=0.2, random_state=42)
    val_imgs, test_imgs = train_test_split(test_imgs, test_size=0.5, random_state=42)

    for split, img_list in zip(["train", "validation", "test"], [train_imgs, val_imgs, test_imgs]):
        split_class_path = os.path.join(SPLIT_DIR, split, folder)
        os.makedirs(split_class_path, exist_ok=True)
        for img in img_list:
            shutil.copy(os.path.join(class_path, img), os.path.join(split_class_path, img))

print("✅ Dataset split completed.")

# 3️⃣ Load Data Using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.2,
                                   height_shift_range=0.2, horizontal_flip=False)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=IMAGE_SIZE,
                                                    batch_size=BATCH_SIZE, class_mode='categorical')

val_generator = val_datagen.flow_from_directory(val_dir, target_size=IMAGE_SIZE,
                                                batch_size=BATCH_SIZE, class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_dir, target_size=IMAGE_SIZE,
                                                  batch_size=BATCH_SIZE, class_mode='categorical')

# 4️⃣ Define CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dropout(0.2),  # Prevent overfitting
    Dense(NUM_CLASSES, activation='softmax')  # Output layer
])

# 5️⃣ Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 6️⃣ Train the Model
history = model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS)


✅ Dataset split completed.
Found 280 images belonging to 7 classes.
Found 35 images belonging to 7 classes.
Found 42 images belonging to 7 classes.
Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 200ms/step - accuracy: 0.1333 - loss: 1.9659 - val_accuracy: 0.1429 - val_loss: 1.9454
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 172ms/step - accuracy: 0.1009 - loss: 1.9474 - val_accuracy: 0.1429 - val_loss: 1.9457
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 174ms/step - accuracy: 0.1723 - loss: 1.9460 - val_accuracy: 0.1429 - val_loss: 1.9454
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 172ms/step - accuracy: 0.1296 - loss: 1.9468 - val_accuracy: 0.1429 - val_loss: 1.9449
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 180ms/step - accuracy: 0.1737 - loss: 1.9443 - val_accuracy: 0.1429 - val_loss: 1.9426
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 176ms/step - accuracy: 0.1223 - loss: 1.9450 - val_accuracy: 0.2857 - val_loss: 1.9385
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 173ms/step - accuracy: 0.1476 - loss: 1.9401 - val_accura

In [23]:
# 7️⃣ Evaluate on Test Set
test_loss, test_acc = model.evaluate(test_generator)
print(f"🔍 Test Accuracy: {test_acc:.4f}") 


# 8️⃣ Save the Model
model.save("gesture_model.h5")
print("✅ Model training complete! Saved as 'gesture_model.h5'")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9474 - loss: 0.1687


🔍 Test Accuracy: 0.9524
✅ Model training complete! Saved as 'gesture_model.h5'
